In [1]:
import cv2
import numpy as np
import os

In [2]:
# show img for testing
def show_image_by_OpenCV(img):
    cv2.imshow('My Image', img )
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [4]:
def detect_Harris_corner(image):
    # parameters
    KERNEL_SIZE = 5
    THRESHOLD = 4000
    K = 0.04

    #  1. Compute x and y derivatives of image.
    # rgb to grayscale
    Gray_img = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) 
    # gaussion : to do small smooth
    Gau_img = cv2.GaussianBlur(Gray_img, (3,3),3)
    # gradient
    I_x = np.gradient(Gau_img, axis=1) 
    I_y = np.gradient(Gau_img, axis=0) 


    # 2. Compute products of derivates at each pixel.
    I_xx = I_x * I_x
    I_yy = I_y * I_y
    I_xy = I_x * I_y

    # 3. Compute the sums of the products of derivates at each pixel.
    # weight : use gaussian
    S_xx = cv2.GaussianBlur(I_xx, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    S_yy = cv2.GaussianBlur(I_yy, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    S_xy = cv2.GaussianBlur(I_xy, (KERNEL_SIZE,KERNEL_SIZE),KERNEL_SIZE)
    
    # 4. Define the matrix M.
    # | S_xx  S_xy |
    # | S_xy  S_yy |

    # 5. Compute the response of the detector at each pixel
    detM = S_xx * S_yy - S_xy * S_xy
    traceM = S_xx + S_yy
    R = detM - K * (traceM * traceM)

    # 6. Threshold on value of R
    R[R < THRESHOLD] = 0
    R[R > 0] = 255
    
    # return images
    return R
        

In [5]:
a = load_images_from_folder('img/csie')
R = detect_Harris_corner(a[0])

In [6]:
show_image_by_OpenCV(R)